In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import imblearn
import collections

import sklearn
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report,confusion_matrix

from keras.utils import to_categorical

from keras.utils.vis_utils import plot_model

from collections import Counter
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN

import pickle

Load the Data

In [2]:
csv_file = 'TStellaULorenzo_alldata_final_doaction.csv'
dataframe = pd.read_csv(csv_file)

In [3]:
dataframe.head()

print(dataframe["Time Since Last Charge"].mean())
print(dataframe["Time Since Last Action Formatted"].mean())

18.594613351404924
1.6497331074212502


Format and One-hot encode the data

In [4]:
dataframe.drop("tsla1", axis=1, inplace=True)
dataframe.drop("tsla2", axis=1, inplace=True)
dataframe.drop("tsla3", axis=1, inplace=True)
dataframe.drop("tsla4", axis=1, inplace=True)
dataframe.drop("tsla5", axis=1, inplace=True)
dataframe.drop("tsla6", axis=1, inplace=True)
dataframe.drop("st1", axis=1, inplace=True)
dataframe.drop("st2", axis=1, inplace=True)
dataframe.drop("st3", axis=1, inplace=True)
dataframe.drop("st4", axis=1, inplace=True)
dataframe.drop("st5", axis=1, inplace=True)
dataframe.drop("st6", axis=1, inplace=True)
dataframe.drop("st7", axis=1, inplace=True)
dataframe.drop("st8", axis=1, inplace=True)
dataframe.drop("st9", axis=1, inplace=True)
dataframe.drop("st10", axis=1, inplace=True)
dataframe.drop("Session Time", axis=1, inplace=True)
dataframe.drop("Time Since Last Action", axis=1, inplace=True)
dataframe.drop("Action", axis=1, inplace=True)

dataframe.drop("Session Time Formatted", axis=1, inplace=True)
#dataframe.drop("Time Since Last Charge", axis=1, inplace=True)  

In [5]:
dataframe.head(100)

,Do Action,Last Action,Time Since Last Charge,Time Since Last Action Formatted,Camera Mood,Gaze,Noise,Input Mood,Activity
0,1,17,0.00,0.02,Happy,1,Low,Negative,Studying
1,0,2,0.00,0.00,Happy,1,None,Negative,Studying
2,0,2,0.00,0.00,Happy,1,Low,Negative,Studying
3,0,2,0.00,0.00,Happy,1,None,Negative,Studying
4,0,2,0.01,0.00,Happy,1,None,Negative,Studying
...,...,...,...,...,...,...,...,...,...
95,0,2,0.31,0.30,Happy,1,None,Negative,Studying
96,0,2,0.31,0.32,Happy,1,None,Negative,Studying
97,0,2,0.31,0.32,Happy,1,Low,Negative,Studying
98,0,2,0.31,0.32,Happy,1,Low,Negative,Studying


In [6]:
dataframe = pd.get_dummies(dataframe, columns=["Activity"], prefix=["Activity"])
dataframe = pd.get_dummies(dataframe, columns=["Camera Mood"], prefix=["Camera_Mood"])
dataframe = pd.get_dummies(dataframe, columns=["Input Mood"], prefix=["Input Mood"])
#both = pd.concat([dataframe, dum_df])
#dataframe.drop("Input Mood", axis=1, inplace=True) 
#both.head(5)

dataframe = pd.get_dummies(dataframe, columns=["Noise"], prefix=["Noise"])
dataframe = pd.get_dummies(dataframe, columns=["Last Action"], prefix=["Last Action"])
dataframe.head(100)

,Do Action,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Angry,Camera_Mood_Happy,Camera_Mood_Neutral,...,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
0,1,0.00,0.02,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0.00,0.00,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.00,0.00,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.00,0.00,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0.01,0.00,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0.31,0.30,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0.31,0.32,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0.31,0.32,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0.31,0.32,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X = dataframe.drop('Do Action', axis=1)
y = dataframe['Do Action']

In [8]:
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
X_train = X[0:55960]
X_test = X[55960:73775]
y_train = y[0:55960]
y_test = y[55960:73775]

In [9]:
counter = Counter(y_train)
print(counter)

Counter({0: 55738, 1: 222})


In [10]:
counter = Counter(y_test)
print(counter)

Counter({0: 17738, 1: 77})


In [11]:
X_train_scaled = X_train.copy()

col_names = ['Time Since Last Charge', 'Time Since Last Action Formatted']
features = X_train_scaled[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_train_scaled[col_names] = features

filename = 'TStellaULorenzo_scaler1.sav'
#pickle.dump(scaler, open(filename, 'wb'))

In [12]:
scaler2 = pickle.load(open(filename, 'rb'))

X_test_scaled = X_test.copy()

features = X_test_scaled[col_names]
features = scaler2.transform(features.values)
X_test_scaled[col_names] = features

X_test_scaled.head()

,Time Since Last Charge,Time Since Last Action Formatted,Gaze,Activity_Relaxing,Activity_Resting,Activity_Studying,Camera_Mood_Angry,Camera_Mood_Happy,Camera_Mood_Neutral,Camera_Mood_Sad,...,Last Action_8,Last Action_9,Last Action_10,Last Action_11,Last Action_12,Last Action_13,Last Action_14,Last Action_15,Last Action_16,Last Action_17
55960,-1.445639,-0.592169,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
55961,-1.445639,-0.599641,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
55962,-1.444163,-0.599641,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
55963,-1.444163,-0.599641,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
55964,-1.444163,-0.599641,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


OSS Undersampling

In [13]:
# define the undersampling method
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
# transform the dataset
X_train_scaled, y_train = undersample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 20342, 1: 222})


SMOTE ADASYN Oversampling

In [14]:
# define the undersampling method
oversample = ADASYN()
# transform the dataset
X_train_scaled, y_train = oversample.fit_resample(X_train_scaled, y_train)
# summarize the new class distribution
counter = Counter(y_train)
print(counter)

Counter({0: 20342, 1: 20309})


Load and Train the Model

In [15]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh

KNeighborsClassifier(n_neighbors=2)

In [16]:
neigh.fit(X_train_scaled,y_train)

KNeighborsClassifier(n_neighbors=2)

In [25]:
#filename = 'TStellaULorenzo_model1.sav'
#pickle.dump(neigh, open(filename, 'wb'))

In [26]:
#loaded_model = pickle.load(open(filename, 'rb'))
#pred = loaded_model.predict(X_test_scaled)
#pred

array([1, 0, 0, ..., 0, 0, 0])

In [17]:
pred = neigh.predict(X_test_scaled)
pred

array([1, 0, 0, ..., 0, 0, 0])

In [18]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80     17738
           1       0.01      0.39      0.01        77

    accuracy                           0.67     17815
   macro avg       0.50      0.53      0.41     17815
weighted avg       0.99      0.67      0.80     17815



In [19]:
y_test.value_counts()

0    17738
1       77
Name: Do Action, dtype: int64

In [20]:
from collections import Counter
Counter(pred)

Counter({1: 5799, 0: 12016})

In [100]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('predknn2.csv')